In [1]:
import nlp.wordpiece as wp
import pandas as pd
import nltk
from string import punctuation
from collections import defaultdict

In [9]:
text = """
prepara la pasta frolla mescolando 300g di farina 150g di burro 100g di zucchero 1 uovo e un pizzico di sale 
impasta fino a ottenere un composto omogeneo avvolgi in pellicola e lascia riposare in frigo per 30 minuti  
stendi la pasta frolla su una superficie infarinata e rivesti uno stampo da crostata imburrato  
bucherella il fondo con una forchetta e cuoci in forno preriscaldato a 180°c per 20 minuti  
prepara la crema pasticcera mescolando 500ml di latte 4 tuorli 100g di zucchero 40g di farina e una scorza di limone  
cuoci a fuoco lento fino a ottenere una crema densa poi lasciala raffreddare  versa la crema pasticcera 
sulla base della crostata raffreddata  decora con frutta fresca a piacere fragole kiwi mirtilli ecc  
scalda un cucchiaio di gelatina di albicocche e spennella la frutta per lucidarla  
lascia raffreddare in frigo per almeno 1 ora prima di servire
"""
print(len(text.split()))

147


In [10]:
wt = wp.WordPieceTokenizer(text=text, maximum_size=500)

In [11]:
example = wt.score_pairs(normalize=False)
print(example.head()) 

#r  #e    18
#e  #r    16
#t  #a    13
#l  #a    12
d   #i    11
dtype: int64


In [12]:
wt.train(normalize=False)

 57%|█████▋    | 286/500 [00:00<00:00, 2024.62it/s]


In [13]:
tokenized = []
for tokens in wt.corpus:
    tokenized.extend(list(tokens))
print(" | ".join(tokenized))

prepara | la | pasta | frolla | mescolando | 300g | di | farina | 150g | di | burro | 100g | di | zucchero | 1 | uovo | e | un | pizzico | di | sale | impasta | fino | a | ottenere | un | composto | omogeneo | avvolgi | in | pellicola | e | lascia | riposare | in | frigo | per | 30 | minuti | stendi | la | pasta | frolla | su | una | superficie | infarinata | e | rivesti | uno | stampo | da | crostata | imburrato | bucherella | il | fondo | con | una | forchetta | e | cuoci | in | forno | preriscaldato | a | 180°c | per | 20 | minuti | prepara | la | crema | pasticcera | mescolando | 500ml | di | latte | 4 | tuorli | 100g | di | zucchero | 40g | di | farina | e | una | scorza | di | limone | cuoci | a | fuoco | lento | fino | a | ottenere | una | crema | densa | poi | lasciala | raffreddare | versa | la | crema | pasticcera | sulla | base | della | crostata | raffreddata | decora | con | frutta | fresca | a | piacere | fragole | kiwi | mirtilli | ecc | scalda | un | cucchiaio | di | ge

In [14]:
V = wt.fancy_vocabulary
V.sort_values(ascending=True)

uovo          1
buchere       1
buch          1
imburrato     1
imburra       1
             ..
#o           51
#i           56
#r           57
#e           58
#a           88
Length: 335, dtype: int64

In [15]:
test = "per preparare la pasta frolla mescola 150g di farina"
print(wt.tokenize_text(test))

['per', 'prepara', '#re', 'la', 'pasta', 'frolla', 'mescola', '150g', 'di', 'farina']


In [10]:
test = "per eccellere nello sport serve molto allenamento"
print(wt.tokenize_text(test))

['per', 'ecc', '#e', '#l', '#l', '#ere', '[UNK]', 'sp', '#or', '#t', 'serv', '#e', 'm', '#o', '#l', '#to', 'al', '#l', '#en', '#a', '#m', '#en', '#to']


## Process explained

In [16]:
text = """
prepara la pasta frolla mescolando 300g di farina 150g di burro 100g di zucchero 1 uovo e un pizzico di sale 
impasta fino a ottenere un composto omogeneo avvolgi in pellicola e lascia riposare in frigo per 30 minuti  
stendi la pasta frolla su una superficie infarinata e rivesti uno stampo da crostata imburrato  
bucherella il fondo con una forchetta e cuoci in forno preriscaldato a 180°c per 20 minuti  
prepara la crema pasticcera mescolando 500ml di latte 4 tuorli 100g di zucchero 40g di farina e una scorza di limone  
cuoci a fuoco lento fino a ottenere una crema densa poi lasciala raffreddare  versa la crema pasticcera 
sulla base della crostata raffreddata  decora con frutta fresca a piacere fragole kiwi mirtilli ecc  
scalda un cucchiaio di gelatina di albicocche e spennella la frutta per lucidarla  
lascia raffreddare in frigo per almeno 1 ora prima di servire
"""
example = "ottenere un"
current = ""
normalize = False 
wt = wp.WordPieceTokenizer(text=text, maximum_size=500)
iterations = list(range(wt.max_vocabulary_size))
for i in iterations:
    check = wt.update(normalize=normalize)
    tokens = wt.tokenize_text(example)
    if tokens != current:
        print("Iterazione {}: {}".format(i, " ".join(tokens)))
        current = tokens
    if not check:
        break 

Iterazione 0: o #t #t #e #n #e #re u #n
Iterazione 3: o #t #t #e #n #er #e u #n
Iterazione 6: o #t #t #en #er #e u #n
Iterazione 8: o #t #t #en #er #e un
Iterazione 20: o #t #t #en #ere un
Iterazione 39: o #t #ten #ere un
Iterazione 89: o #t #tenere un
Iterazione 91: o #ttenere un
Iterazione 107: ottenere un


In [17]:
len(wt.vocabulary)

335

In [13]:
for w, s in wt.fancy_vocabulary.items():
    if w.startswith('o'):
        print(w, s)

o 4
ottenere 2
om 1
omo 1
omog 1
omogen 1
omogene 1
omogeneo 1
ora 1


In [20]:
from transformers import BertTokenizer

In [21]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-uncased")

## BERT Tokenizer

In [23]:
example = "donna donne uomo uomini"

tokens = tokenizer.tokenize(example)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print("Testo originale:", example)
print("Token:", tokens)
print("Token IDs:", token_ids)

Testo originale: donna donne uomo uomini
Token: ['donna', 'donne', 'uomo', 'uomini']
Token IDs: [1783, 2017, 1735, 1738]
